---

# Running the experiment on Google Colab


In [0]:
from torchvision import transforms
from google.colab import drive
from torch.utils import data
from PIL import Image

import matplotlib.pyplot as plt
import PIL.Image as pil_image
import torch.nn as nn
import torchvision
import numpy as np
import torch
import time
import cv2
import os

In [16]:
drive.mount('/content/gdrive')
path ='/content/gdrive/My Drive/deep_learning_group_7/Final'
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
from IPython.display import HTML
HTML(filename = path + '/interactive_image.html')

# ESPCN Architecture


In [0]:
r = 3 #scaling factor

class SuperResConvNet(nn.Module):
  def __init__(self):
    'Initialize layers'
    super(SuperResConvNet, self).__init__()
    self.conv1 = nn.Conv2d(1,64, kernel_size=5, stride=1, padding=2)
    self.conv2 = nn.Conv2d(64,32, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(32,1*r**2, kernel_size=3, stride=1, padding=1)
    self.upsample = nn.PixelShuffle(r)

  def forward(self, y):
    'Define forward pass'
    y = torch.tanh(self.conv1(y))
    y = torch.tanh(self.conv2(y))
    y = self.conv3(y)
    y = self.upsample(y)
    return y

# Check for GPU availibility
if torch.cuda.is_available():
  print("Using GPU")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define model and send to device
ConvNet = SuperResConvNet()
ConvNet.to(device)

Using GPU


SuperResConvNet(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (upsample): PixelShuffle(upscale_factor=3)
)

#Functions

In [0]:
# Define directories
slide_subfolders = ['yang91/T91/', #trainingset
                    'x3.0/Set5/', 'x3.0/Set14/', # Testsets
                    'x3.0/BSD300/', 'x3.0/BSD500/', 'x3.0/SuperText136/'] 

#width and height of patches
x = 17      

class Slide:
  'Combines functions for loading and preparing images for training and testing'
  def __init__(self, path, slide_subfolders, count):
    self.count = count
    self.dir = path + '/CVPR2016_ESPCN_OurBenchMarkResult/Ours/' + slide_subfolders
    
    self.namelist = self.deleteLRImage()
    if count == 0:
        self.patchlist = self.patchlist_get()
      
  def removePng(f):
    'returns filename without png'
    filename_parts = f[:-4]
    return filename_parts

  def getList(self):
    'returns list of filenames' 
    return [Slide.removePng(f) for f in os.listdir(self.dir) if f.endswith(".png")]

  def deleteLRImage(self):
    'deletes LR images from list if they are already made (which they are)'
    name_list_2 = Slide.getList(self)
    name_list_1 = [x for x in name_list_2 if "lr" not in x ]
    name_list =  [x for x in name_list_1 if "lowRes" not in x ]
    return name_list
  
  def getPatchList(self, img_name):
    'returns patch list of one image'
    #get filenames and load images
    filename = self.dir + img_name
    lr_img = cv2.cvtColor(cv2.imread(filename+'_lr.png'), cv2.COLOR_BGR2RGB)

    #parameters 
    stride_lr = x-np.sum((5%2,3%2,3%2))
    tot_img_d = int(lr_img.shape[0]/stride_lr), int(lr_img.shape[1]/stride_lr)    #amount of image in height and width respectively
    tot_img = tot_img_d[0]*tot_img_d[1]     #total amount of images

    #create list for current image
    patch_list = []
    for i in range(tot_img_d[0]-1):
        for j in range(tot_img_d[1]-1):
            patch_list.append([img_name, i,j])
    return patch_list
  
  def patchlist_get(self):
    'create patch_list'
    patch_list = []
    for i in range(len(self.namelist)):
        patch_list.extend(self.getPatchList(self.namelist[i]))
    print('Found', len(patch_list), 'trainable patches out of', len(self.namelist), 'images.')
    return patch_list
  
  def createPatch(self, name):
    'returns a patch'
    img_name = name[0]
    patch_name = name[1], name[2]

    #get corresponding images
    filename = self.dir + img_name
    hr_img = cv2.cvtColor(cv2.imread(filename+'.png'), cv2.COLOR_BGR2YCrCb)[:,:,0] # only get Y channel from YCrCb
    lr_img = cv2.cvtColor(cv2.imread(filename+'_lr.png'), cv2.COLOR_BGR2YCR_CB)[:,:,0]

    #create hr patch 
    stride_hr = (x-np.sum((5%2,3%2,3%2)))*r
    hr_patch = hr_img[stride_hr*patch_name[0]:(stride_hr*patch_name[0]+17*r),stride_hr*patch_name[1]:(stride_hr*patch_name[1]+17*r)]

    #create lr patch 
    stride_lr = x-np.sum((5%2,3%2,3%2))
    lr_patch = lr_img[stride_lr*patch_name[0]:(stride_lr*patch_name[0]+17),stride_lr*patch_name[1]:(stride_lr*patch_name[1]+17)]

    return lr_patch, hr_patch

  def bicubic_upsampling(self, img_n):
    'Loading high and low resolution image and do a bicubic upsampling of the low resolution image'
    #get corresponding images
    filename = self.dir + img_n
    hr_img = Image.open(filename + '.png').convert('YCbCr')
    lr_img = Image.open(filename[:-9] + '-lowRes.png').convert('YCbCr')

    #upsample
    bicubic = lr_img.resize(hr_img.size, Image.BICUBIC)

    return lr_img, bicubic, hr_img # return as pil images

def getSlideList(slide_subfolders, path):
  'Load Slide class for different directories'
  slides = []
  for i, slide in enumerate(slide_subfolders):
    slides.append(Slide(path, slide, i))
  return slides

slides = getSlideList(slide_subfolders, path)

Found 1734 trainable patches out of 91 images.


### PSNR calculation

In [0]:
def psnr_from_mselist(mse_list):
  'Calculate PSNR'
  mse = np.mean(mse_list)
  if mse == 0:
    return float('inf')
  else:
    return 20*np.log10(255/np.sqrt(mse))

def calc_mse(img_pred, img_hr):
  'Calculate MSE'
  return np.mean((img_pred*255 - img_hr*255)**2)

# Obtaining low resolution images

In [0]:
## ONLY RUN THIS CELL IF LOW RESOLUTION IMAGES ARE NOT PRESENT IN THE DIRECTORY
def createLowRes(img_name, dir_91):
  'saves low resolution image'
  # Call HR image
  filename = dir_91 + img_name + '.png'
  hr_img = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)

  # Blur HR image
  blur_img = cv2.GaussianBlur(hr_img,(5,5),0)

  # Apply subsampling
  lr_img = blur_img[::r,::r]

  # Save lr_img
  im = Image.fromarray(lr_img)
  im.save(dir_91 + img_name + '_lr.png')

for i in range(len(slides[0].namelist)):
  dir_91 = path + '/CVPR2016_ESPCN_OurBenchMarkResult/Ours/' + slide_subfolders[0]
  createLowRes(slides[0].namelist[i], dir_91)

---
# Training


##Hyperparameter settings

In [0]:
# Loss and optimizer
criterion = nn.MSELoss()     
optimizer = torch.optim.Adam(ConvNet.parameters(), lr=0.01)  

# Parameters
num_epoch = 5000        #Amount of epochs
batch_size = 16         #Batch size
train_val_ratio = 0.95  #Training validation ratio

# Scheduler for dynamic reduction of the learning rate
threshold_mu = 1e-6     # Treshold for decreasing learning rate.
factor_value = 0.8     # Amount of decay per step, new lr = factor_value*lr.
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode='min',
                                                       factor=factor_value,
                                                       patience=2,
                                                       threshold=threshold_mu,
                                                       min_lr=0.0001,
                                                       eps=1e-08,
                                                       verbose=True)

--- 
### Data loader

In [0]:
class DataGenerator(data.Dataset):
  'Generates the dataset that is used for training the ESPCN'
  def __init__(self, slides):
    self.slides = slides
    self.transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()])

  def __len__(self):
    'Returns the amount of patches'
    return len(self.slides.patchlist)

  def __getitem__(self, idx):
    'Returns low and high resolution patches in the form of tensors'
    lr_patch, hr_patch = self.slides.createPatch(self.slides.patchlist[idx])
    
    # transform images to pytorch tensors
    lr_patch = self.transform(lr_patch)
    hr_patch = self.transform(hr_patch)

    return lr_patch, hr_patch   

# Put DataGenerator in DataLoader
full_dataset = DataGenerator(slides[0])

# Split between training and validation set
train_size = int(train_val_ratio * len(full_dataset))
validation_size = len(full_dataset) - train_size
training_set, validation_set = torch.utils.data.random_split(full_dataset, [train_size, validation_size])

# Create training and validation data loaders
training_generator      = data.DataLoader(training_set, batch_size=batch_size, num_workers=batch_size, shuffle='True')
validation_generator    = data.DataLoader(validation_set, batch_size=1, num_workers=1, shuffle='False')

### Training the model

In [0]:
# Initializing lists used for saving losses and PSNR
loss_list = []
epoch_loss_list = []
val_loss_list = []
validation_psnr = []

# Start timer
t0 = time.time()

# Training loop
for epoch in range(num_epoch):
  # Switch to training mode
  ConvNet.train()
  for i, (lr_patch, hr_patch) in enumerate(training_generator):
      
    # Transfer training data to active device
    lr_patch, hr_patch = lr_patch.to(device), hr_patch.to(device)
    
    # Run the forward pass
    outputs = ConvNet(lr_patch)
    loss = criterion(outputs, hr_patch)
    loss_list.append(loss.item())

    # Backprop and perform Adam optimisation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Save loss every epoch
  epoch_loss = np.sum(loss_list)/len(training_generator)
  epoch_loss_list.append(epoch_loss)

  # Print epoch loss every 50 epochs
  if epoch % 50 == 0:
    print("Epoch", epoch, "loss: {}".format(epoch_loss))
  
  # Save model every 1000 epochs (in case Google Colab stops runtime)
  # if epoch % 1000 == 999:
  #   model_name = 'final_' + str(epoch+1) + '_epochs'
  #   path_model = path + '/saved_models/' + model_name
  #   torch.save(ConvNet.state_dict(), path_model)
  #   print('Model saved as: ', model_name)

  # Step to next step of lr-scheduler
  scheduler.step(epoch_loss)
  loss_list = []

  # Enter validation mode
  ConvNet.eval()

  # Keep track of mse for every patch, to collectively calculate PSNR per epoch
  epoch_mse = []
  with torch.no_grad():
    for i, (lr_patch, hr_patch) in enumerate(validation_generator):
      # Transfer training data to active device (GPU)
      lr_patch, hr_patch = lr_patch.to(device), hr_patch.to(device)

      # Predict output
      img_pred = ConvNet(lr_patch)
      
      # Calculate validation loss
      loss = criterion(img_pred, hr_patch)
      loss_list.append(loss.item())
      
      # Calculate mse for every sample 
      img_pred = img_pred[0].cpu().numpy()
      hr_patch = hr_patch[0].cpu().numpy()
      epoch_mse.append(calc_mse(img_pred, hr_patch))

    # Calculate validation psnr on the complete epoch from all individual MSE's
    val_psnr = psnr_from_mselist(np.array(epoch_mse))
    validation_psnr.append(val_psnr)
  
  # Save validation loss every epoch
  val_loss = np.sum(loss_list)/len(validation_generator)
  val_loss_list.append(val_loss)
  
  loss_list = []

print('Training took {} seconds'.format(time.time() - t0))
print('Seconds per epoch:',(time.time()-t0)/num_epoch)

### Plot training results

In [0]:
# Show training and validation loss of current model in memory
plt.figure(figsize=(8,8))
plt.subplot(2,1,1)
plt.title('Loss per epoch')
plt.plot(np.arange(num_epoch), epoch_loss_list, label='Training loss')
plt.plot(np.arange(num_epoch), val_loss_list, label='Validation loss')
plt.yscale("log")
plt.legend()

plt.subplot(2,1,2)
plt.title('PSNR for validation data')
plt.plot(np.arange(num_epoch), validation_psnr)

plt.show()

# Testing


#### Loading the model

In [0]:
# Load weight in earlier defined model
model_name = 'final_test_5000_epochsweights'
path_model = path + '/saved_models/' + model_name
ConvNet.load_state_dict(torch.load(path_model))

<All keys matched successfully>

###Test image plot function

In [0]:
def generate_figure(lr_img, sr_img, hr_img):
  'Show the low resolution, upscaled and high resolution image'
  f = plt.figure(figsize=(8*3,8))
  f.add_subplot(1, 3, 1)
  plt.imshow(transforms.ToPILImage('YCbCr')(lr_img[0]).convert('RGB'))
  f.add_subplot(1, 3, 2)
  plt.imshow(transforms.ToPILImage('YCbCr')(sr_img[0]).convert('RGB'))
  f.add_subplot(1, 3, 3)
  plt.imshow(transforms.ToPILImage('YCbCr')(hr_img[0]).convert('RGB')) 

####Data loader

In [0]:
class DataGenerator_test(data.Dataset):
  'Generates the dataset used for testing'

  def __init__(self, slides):
    self.slides = slides
    self.transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

  def __len__(self):
    'Returns the amount of test images'
    return len(self.slides.namelist)

  def __getitem__(self, idx):
    'Returns low, upsampled and high resolution testing images in the form of tensors'
    lr_img, bicubic, hr_img = self.slides.bicubic_upsampling(self.slides.namelist[idx])
    
    lr_img = self.transform(lr_img)
    hr_img = self.transform(hr_img)
    bicubic = self.transform(bicubic)

    return lr_img, bicubic, hr_img

## Put DataGenerator in DataLoader
def DataGenerator(slides):
  'Create dataloader for every test directory'
  test_set = DataGenerator_test(slides)
  test_generator = data.DataLoader(test_set, batch_size=1, shuffle=False)
  return test_generator

###Testing the model

In [0]:
import random as rnd
rnd.seed(9)

# Loop over test data sets
for j in range(len(slides[1:])):

  # Load data
  test_generator = DataGenerator(slides[j+1])

  show = rnd.randint(0,len(test_generator))

  # Define lists
  outputs = []
  mse_list = []
  time_list = []

  # Main testing loop
  for i, (lr_img, bicubic, hr_img) in enumerate(test_generator):
    with torch.no_grad():
        
      # Only test on the Y (intensity channel)
      to_network = (lr_img[:,0,:,:]).unsqueeze(0)

      # Start timer
      start_time = time.time()
      
      # Run the forward pass
      outputs = ConvNet.cpu()(to_network)
      img_pred = outputs[0]

      # Substitute the Y channel from bicubic with the one outputted by the model
      bicubic[:,0,:,:] = img_pred

      #Stop timer
      elapsed_time = time.time() - start_time
      time_list.append(elapsed_time)

      # Calculate MSE
      mse = calc_mse(img_pred.numpy(), hr_img.numpy()[:,0,:,:])
      mse_list.append(mse.item())

      #Show one random image from every dataset
      if i == show:
        generate_figure(lr_img, bicubic, hr_img)

  # Printing the results
  print(slide_subfolders[j+1], ': PSNR', psnr_from_mselist(mse_list), 'Average time' ,np.mean(time_list))

Output hidden; open in https://colab.research.google.com to view.